In [1]:
import random
import os
import re
import numpy as np
from pprint import pprint
print(os.getcwd())
from Backend.py_utils import *
from Backend import perf_calc as perf_calc
from fastapi import FastAPI, Request
from fastapi.responses import RedirectResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from Backend import perf_calc as perf_calc
from Backend import rotation_calc as rotation_calc
from Backend import refusal as refusal
from database import database_handler
from fastapi.responses import JSONResponse
from Backend import handle_route as handle_route
from pydantic import BaseModel 
from Backend import wind_calc
import sqlite3

g:\CompSciProject
g:\CompSciProject
g:\CompSciProject
g:\CompSciProject


In [11]:
def search_airport(partial_name):
    database_path = "database/nav_data.db"
    connect_to_db = sqlite3.connect(database_path)
    cursor = connect_to_db.cursor()

    words = partial_name.split()
    conditions = " AND ".join(["name LIKE ? COLLATE NOCASE" for _ in words])

    query = f"""
    SELECT name, icao,type
    FROM airports 
    WHERE {conditions} 
    ORDER BY 
        LENGTH(name), 
        INSTR(LOWER(name), LOWER(?)) 
    """

    params = tuple(f"%{word}%" for word in words) + (words[0],)

    cursor.execute(query, params)
    results = cursor.fetchall()
    connect_to_db.close()

    return results
search_airport("hello")

[('Othello Municipal Airport', 'KS70', 'small_airport')]

In [ ]:
from math import cos, asin, sqrt, pi
import sqlite3
def nearby_points(point_lat, point_lng, area_code):
    database_path = "database/nav_data.db"
    connect_to_db = sqlite3.connect(database_path)
    cursor = connect_to_db.cursor()

    cursor.execute("SELECT waypoint_identifier, waypoint_latitude, waypoint_longitude FROM waypoints WHERE area_code = ?", (area_code,))

    close_wp = []

    data = cursor.fetchall()

    def distance(lat1, lon1, lat2, lon2):
        r = 3958.8 # miles
        p = pi / 180

        a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
        return 2 * r * asin(sqrt(a))
    
    for waypoint in data:
        waypoint_id, waypoint_lat, waypoint_lng = waypoint
        dist = distance(point_lat, point_lng, waypoint_lat, waypoint_lng)
       
        if dist < 20:
            close_wp.append([waypoint_id,waypoint_lat,waypoint_lng, dist])
    
    return close_wp

nearby_points(51.68527778,0.68333333,"EUR")

In [5]:
def store_route(route, username):

    freeslot = None
    taken_slots = set() # needs to be an empty set not "None"

    conn = sqlite3.connect("database/users.db")
    cursor = conn.cursor()
    cursor.execute("SELECT route_number FROM user_routes WHERE username = ?", (username,)) # gets all taken slots in db for that user
    for row in cursor.fetchall():
        taken_slots = taken_slots + row
        print(f"row: {row}")
        print(f"taken slots: {taken_slots}") # puts all taken slots in a variable

    for slot in range(1,11):
        if slot not in taken_slots:
            freeslot = slot # finds the next free slot
            print(f"free slot is: {freeslot}")
            break # escape loop

    if freeslot == None: # if there is no free spaces
        print("NO FREE SLOT FOUND IN DB TO SAVE ROUTE")

    cursor.execute("INSERT INTO user_routes (username, route_number, route_data) VALUES (?,?,?)", (username, freeslot, route)) # put the data in 

    conn.commit()
    conn.close()
    
    return {"message": "Route saved", "slot": freeslot}

store_route("EGLL POLIG EGMC", "Tommyboy0797")

free slot is: 1


{'message': 'Route saved', 'slot': 1}

In [6]:
def get_saved_routes(username):
    conn = sqlite3.connect("database/users.db")
    cursor = conn.cursor()
    cursor.execute("SELECT route_number, route_data FROM user_routes WHERE username = ? ORDER BY route_number", (username,))
    rows = cursor.fetchall()
    conn.close()

    routes = []
    for route_number, route_data in rows:
        route_parts = route_data.strip().split()
        from_airport = route_parts[0] if len(route_parts) > 0 else ""
        to_airport = route_parts[-1] if len(route_parts) > 1 else ""

        routes.append({
            "route_name": f"Route {route_number}",
            "from": from_airport,
            "to": to_airport,
            "last_used": "Just now",  # update later with timestamb
            "route_data": route_data
        })

    return {"routes": routes}

get_saved_routes("Tommyboy0797")

{'routes': [{'route_name': 'Route 1',
   'from': 'EGLL',
   'to': 'EGMC',
   'last_used': 'Just now',
   'route_data': 'EGLL POLIG EGMC'},
  {'route_name': 'Route 2',
   'from': 'EINN',
   'to': 'EHAM',
   'last_used': 'Just now',
   'route_data': 'EINN RW24 PELI2B MONTY OTBED MOLI2A RW18R EHAM'},
  {'route_name': 'Route 3',
   'from': 'EINN',
   'to': 'EHAM',
   'last_used': 'Just now',
   'route_data': 'EINN RW24 MOMI2B ROBVA LECND NORK2A RW18C EHAM'},
  {'route_name': 'Route 4',
   'from': '',
   'to': '',
   'last_used': 'Just now',
   'route_data': ''},
  {'route_name': 'Route 5',
   'from': '',
   'to': '',
   'last_used': 'Just now',
   'route_data': ''}]}

In [ ]:
def remove_route(username, routename):
    route_number = int(routename.split()[1])
    
    conn = sqlite3.connect("database/users.db")
    cursor = conn.cursor()

    cursor.execute("DELETE FROM user_routes WHERE username = ? AND route_number = ?", (username, route_number))

    conn.commit()
    conn.close()

    return {"status": "success", "message": f"Route {route_number} deleted."}

remove_route("Tommyboy0797", "Route 5")

{'status': 'success', 'message': 'Route 4 deleted.'}

In [7]:
class SendString(BaseModel):
    send_str:str

def return_runway(runwy: SendString, origin: SendString):

    sids = {
        "sids": database_handler.get_sids(origin.send_str, runwy.send_str),
        "runway_data": database_handler.get_runway_data(origin.send_str, runwy.send_str)
    }
    return sids

def handle_stars(selected_runway: SendString, arrival_airfield: SendString):

    star_data = {
        "arrival_stars": database_handler.get_stars(arrival_airfield.send_str, selected_runway.send_str),
    }


    return star_data

def route_data(route: SendString):
    route_list = route.send_str.split() # list of the route parts

    departure = route_list[0] # get first word, departure field
    departure_runway = route_list[1]
    sid = route_list[2]
    is_sid = False

    arrival = route_list[-1] # last word
    arrival_runway = route_list[-2] # second to last word (which is the runway in every case)
    star = route_list[-3]
    is_star = False

    sidsdata = return_runway(SendString(send_str=departure_runway), SendString(send_str=departure))
    starsdata = handle_stars(SendString(send_str=arrival_runway), SendString(send_str=arrival))

    if sid in sidsdata["sids"]: # check if there is a sid
        sid = route_list[2]
        is_sid = True
    else:
        sid = ""
    if star in starsdata["arrival_stars"]: # check if there is a star
        star = route_list[-3]
        is_star = True
    else:
        star = ""   
    waypoints = []
    if is_star and is_sid == True:
        waypoints = route_list[2:-3]
    elif is_star == True and is_sid == False:
        waypoints = route_list[1:-3]
    elif is_star == False and is_sid == True:
        waypoints = route_list[2:-2]
        


    return (f"departure: {departure}, dep rwy: {departure_runway}, SID: {sid}, waypoints: {waypoints}, STAR: {star}, arrival rwy: {arrival_runway}, arrival: {arrival}")

route_data(SendString(send_str="EGMC RW23 RIKPU LAMS2E RW21 EHEH"))

'departure: EGMC, dep rwy: RW23, SID: , waypoints: [], STAR: , arrival rwy: RW21, arrival: EHEH'